# Sequential Chains

The next step after calling a language model is make a series of calls to a language model. This is particularly useful when you want to take the output from one call and use it as the input to another.

In this notebook we will walk through some examples for how to do this, using sequential chains. Sequential chains are defined as a series of chains, called in deterministic order. There are two types of sequential chains:

- `SimpleSequentialChain`: The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.
- `SequentialChain`: A more general form of sequential chains, allowing for multiple inputs/outputs.

## SimpleSequentialChain

In this series of chains, each individual chain has a single input and a single output, and the output of one step is used as input to the next.

Let's walk through a toy example of doing this, where the first chain takes in the title of an imaginary play and then generates a synopsis for that title, and the second chain takes in the synopsis of that play and generates an imaginary review for that play.

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [4]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [5]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [6]:
review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...


Tragedy at sunset on the beach is a heartfelt drama about two couples whose lives and relationships intersect in unexpected ways on a summer evening at the beach. 

The play centers around the couple David and Julia, who are visiting the beach for a romantic evening. As they watch the sun set, they are joined by two unexpected visitors: their friends, John and Sarah. As the four converse, they quickly realize that their relationships and lives are intertwined in ways that they never expected. 

As the evening progresses, secrets are revealed and tensions rise as it becomes clear that David and Sarah had a past relationship. In the end, the four of them must grapple with the consequences of their secrets and the complex emotions that their relationships have created. 

Tragedy at sunset on the beach is a story of love, loss, and the power of secrets. It is a story of how the past can come back to haunt us and how we must confront our cho

In [7]:
print(review)



Tragedy at Sunset on the Beach is a powerful drama that explores the complexity of relationships and the power of secrets. The story follows two couples, David and Julia, and John and Sarah, as their lives and relationships intertwine on a summer evening at the beach. As the sun sets and secrets are revealed, tensions rise as the four must grapple with the consequences of their choices and their past.

The writing is gripping and the performances are captivating. The actors bring the relationships to life in an honest and nuanced way, making the characters and their struggles feel real. The audience is taken on an emotional journey as they watch the characters confront their past, their choices, and the complex emotions that their relationships have created.

Tragedy at Sunset on the Beach is a beautifully written and performed drama that will leave audiences thinking long after the curtain has closed. It is a story of love, loss, and the power of secrets that will stay with you long

## Sequential Chain
Of course, not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain. In this next example, we will experiment with more complex chains that involve multiple inputs, and where there also multiple final outputs. 

Of particular importance is how we name the input/output variable names. In the above example we didn't have to think about that because we were just passing the output of one chain directly as input to the next, but here we do have worry about that because we have multiple inputs.

In [8]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", 'era'], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [9]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

In [10]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [11]:
review = overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


### Memory in Sequential Chains
Sometimes you may want to pass along some context to use in each step of the chain or in a later part of the chain, but maintaining and chaining together the input/output variables can quickly get messy.  Using `SimpleMemory` is a convenient way to do manage this and clean up your chains.

For example, using the previous playwright SequentialChain, lets say you wanted to include some context about date, time and location of the play, and using the generated synopsis and review, create some social media post text.  You could add these new context variables as `input_variables`, or we can add a `SimpleMemory` to the chain to manage this context:

In [12]:
from langchain.chains import SequentialChain
from langchain.chains.base import SimpleMemory

llm = OpenAI(temperature=.7)
template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': 'This Christmas, come to Theater in the Park to watch the powerful story of young love in "Fate\'s Tide". The play follows the story of two star-crossed lovers, Sally and John, as they fight against fate to be together. You won\'t want to miss the incredible performances and impressive set design that bring this tragedy to life! Get your tickets now! #FatesTide #YoungLove #TheaterInThePark #ChristmasShow'}